For colab

In [1]:
import os
import sys
from google.colab import drive

# Mount google drive.
drive.mount('/gdrive')

!pip install simpletransformers
!pip install wandb -qU

from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn
import torch
import wandb

Colab=True

Mounted at /gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#  Apex Library install
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

!sh setup.sh

wandb.login()

In [2]:
#Query the current cuda version
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
#For Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device:', device)

Using device: cuda


## Dataset prep

Final Adjustments to the dataset before training. Some outputs are removed to protect Propriety information

In [ ]:
pd.set_option('display.max_columns', None)
if Colab==False:
    train_df=pd.read_csv('path to/train_data.csv', encoding='ISO-8859-1')
    test_df=pd.read_csv('path to/test_data.csv', encoding='ISO-8859-1')
    val_df=pd.read_csv('path to/val_data.csv', encoding='ISO-8859-1')
if Colab==True:
    train_df=pd.read_csv('path to/train_data.csv', encoding='ISO-8859-1')
    test_df=pd.read_csv('path to/test_data.csv', encoding='ISO-8859-1')
    val_df=pd.read_csv('path to/val_data.csv', encoding='ISO-8859-1')

train_df.head()

In [ ]:
train_df['Object Text'] = train_df['Object Text'].astype('str')
test_df['Object Text'] = test_df['Object Text'].astype('str')
val_df['Object Text'] = val_df['Object Text'].astype('str')

train_df_binary=train_df.drop(['Discipline','Module Name', 'Module UID', 'Requirement ID'],axis=1)
test_df_binary=test_df.drop(['Discipline','Module Name', 'Module UID', 'Requirement ID'],axis=1)
val_df_binary=val_df.drop(['Discipline','Module Name', 'Module UID', 'Requirement ID'],axis=1)

train_df_binary = train_df_binary.rename(columns={"Object Text": "text", "Object Type": "labels"})
test_df_binary = test_df_binary.rename(columns={"Object Text": "text", "Object Type": "labels"})
val_df_binary = val_df_binary.rename(columns={"Object Text": "text", "Object Type": "labels"})
train_df_binary.head()

In [6]:
train_df_binary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9542 entries, 0 to 9541
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    9542 non-null   object
 1   labels  9542 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 149.2+ KB


## Configuring the model

In [7]:
sweep_config = {
    "method": "grid",  # grid, random, bayes
    "metric": {"name": 'eval_loss', "goal": 'minimize'},
    "parameters": {
        "optimizer": {"values": ['AdamW', 'Adafactor']},
        "train_batch_size" :{"values": [32, 8, 4]},
        "learning_rate": {"values": [4e-5, 1e-5]}
        #"learning_rate": {"min": 5e-5, "max": 4e-4},
    },
    "early_terminate" : {
        "type": "hyperband",
        "eta": 2,
        "min_iter":2
    }
}

sweep_id = wandb.sweep(sweep_config, project="ObjectTypeRobertaV1.01")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: cui4eq6j
Sweep URL: https://wandb.ai/alanpaddy/ObjectTypeRobertaV1.01/sweeps/cui4eq6j


In [8]:
binary_args={
    'output_dir':'/gdrive/MyDrive/Transformers/V2/ObjectType/Checkpoints/', #where all outputs will be stored. This includes model checkpoints and evaluation results.
    'best_model_dir':'/gdrive/MyDrive/Transformers/V2/ObjectType/BestModel/', #The directory where the best model (model checkpoints) will be saved (based on eval_during_training)
    'cache_dir': '/gdrive/MyDrive/Transformers/V2/ObjectType/Cache/', #The directory where cached files will be saved.
    'tensorboard_dir': '/gdrive/MyDrive/Transformers/V2/ObjectType/TensorBoard/', 
    'overwrite_output_dir': True, # trained model will be saved to the ouput_dir but will overwrite existing saved models in the same directory.
    'reprocess_input_data': True, # input data will be reprocessed even if a cached file of the input data exists in the cache_dir.
    'train_batch_size':32, #Larger batch sizes train faster but consume more memory and might show lower accuracy
    'eval_batch_size': 8, #Evaluation batch size
    'gradient_accumulation_steps':1, #The number of training steps to execute before performing a optimizer.step(). Effectively increases the training batch size while sacrificing training time to lower memory consumption.
    'learning_rate': 4e-5,#Lower LR increases accuracy but also increases training time.
    'adafactor_relative_step': True,# If True, time-dependent learning rate is computed instead of external learning rate.
    'num_train_epochs': 4, #3 passes through the training set
    'show_running_loss': True, #Log the loss to the console
    'max_seq_length': 128, #512 #Max number of input tokens
    'weight_decay': 0, #Prevents overfitting by penalizing large weights via adding the L2 penalty.
    'adam_epsilon': 1e-8, #Epsilon parameter used in AdamOptimizer. AdamOptimizer is the extended version of SGD.
    'warmup_ratio': 0.06, #Ratio of total training steps where learning rate will “warm up”.The learning rate is increased linearly over the warm-up period.
    'warmup_steps': 0,#Overrides warmup_ratio if not zero.
    'max_grad_norm': 1.0, #Maximum Gradient clipping. Avoids exploding gradients.
    'logging_steps': 1, #Log training loss and learning at every specified number of steps
    'evaluate_during_training': True, #perform evaluation while training model
    'save_steps': 500, #Save a model checkpoint at every specified number of steps
    'use_early_stopping' : True, #Use early stopping to stop training when early_stopping_metric doesn’t improve (based on early_stopping_patience, and early_stopping_delta)
    'early_stopping_metric': 'eval_loss', #Use validation loss as the early stopping metric
    'early_stopping_patience': 3, #Terminate training after this many evaluations without an improvement in the evaluation metric greater then early_stopping_delta.
    'early_stopping_delta': 0.005, #The improvement over best_eval_loss necessary to count as a better checkpoint.
    #'encoding': #Specify an encoding to be used when reading text files.
    'use_tensorboard': True,
    'optimizer': 'AdamW', #OR Adafactor
    'polynomial_decay_schedule_lr_end': 1e-7, #The end learning rate.
    'scheduler' : 'linear_schedule_with_warmup', #The scheduler to use when training. 
    'fp16': False, #fp16 mode requires NVidia Apex library. Can process up to 8 times faster than FP32 on modern GPUs and require less memory.
    'n_gpu': 1, #Number of GPUs to be used
    'wandb_project':'ObjectTypeRobertaV1.01', #Name of W&B project. This will log all hyperparameter values, training losses, and evaluation metrics to the given project.
    'wandb_kwargs' :{'dir': '/gdrive/MyDrive/Transformers/V2/ObjectType/WandB/' }
}

In [9]:
def train():
    wandb.init(dir='/gdrive/MyDrive/Transformers/Binary/WandB/')

    modelBinary = ClassificationModel('roberta', 'roberta-base', args=binary_args, use_cuda=True)

    modelBinary.train_model(train_df_binary,
                        output_dir='/gdrive/MyDrive/Transformers/V2/ObjectType/Checkpoints/',
                        show_running_loss=True,
                        eval_df=val_df_binary,
                        verbose=True
                        )
    modelBinary.eval_model(val_df_binary)
    wandb.join()

wandb.agent(sweep_id, train, count=14) #Count should really be 12, since there are only 12 combinations. But wandb agent has a known bug of creating duplicates https://github.com/wandb/wandb/issues/3522

wandb: Agent Starting Run: vw0myfc6 with config:
wandb: 	learning_rate: 4e-05
wandb: 	optimizer: AdamW
wandb: 	train_batch_size: 32
wandb: Currently logged in as: alanpaddy. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/9542 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/299 [00:00<?, ?it/s]

...

Running Epoch 1 of 4:   0%|          | 0/299 [00:00<?, ?it/s]

  0%|          | 0/2386 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/299 [00:00<?, ?it/s]

Training loss,█▅▃▆▄▁▂▁▁▁▁▂▁▂▂▂▂▁▁▅▁▂▃▃▃▂▁▁▂▂▂▁▁▁▁▃▃▂▁▁
auprc,▁█
auroc,▁█
eval_loss,█▁
fn,█▁
fp,█▁
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,▁▃▅▇████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃
mcc,▁█
tn,▁█
tp,▁█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [10]:
modelBinaryTrained= ClassificationModel('roberta', '/gdrive/MyDrive/Transformers/V2/ObjectType/BestModel/')
wandb.init() 
result, model_outputs, wrong_predictions = modelBinaryTrained.eval_model(test_df_binary,
                                                            output_dir='/gdrive/MyDrive/Transformers/V2/ObjectType/Results/',
                                                            verbose=True)

  0%|          | 0/2983 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/373 [00:00<?, ?it/s]